In [1]:
import cv2
import os
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt
import time
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose # Holistic model

In [2]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    results=model.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [3]:
def draw_landmarks(image, results):

    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS) # Draw pose connections


In [31]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    if len(pose) != 132:
    # Use numpy.pad() to add zeros at the end
        padded_array = np.pad(pose, (0, 132-len(pose)), 'constant')
    else:
        # No zeros need to be added
        padded_array = pose
    return padded_array


In [5]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [9]:
for action in actions: 
    #dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [32]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(1, start_folder+1):
            # Loop through video length aka sequence length
        
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, pose)
                
                # Draw landmarks
                draw_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screenqqq
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                print("-------KEYPOINTS------------")
                print(keypoints.shape)
                print("-------KEYPOINTS------------")
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS------------
(132,)
-------KEYPOINTS------------
-------KEYPOINTS---------

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
label_map = {label:num for num, label in enumerate(actions)}

In [34]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            if len(res)!=132:
                print("WTF: ",len(res) )
                print(res)
             
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [25]:
print(len(sequences))

90


In [35]:
X = np.array(sequences)
print(X.shape)

[[[ 3.15551490e-01  6.19041979e-01 -1.27203047e+00 ...  3.91091371e+00
    6.22943044e-01  7.19858008e-06]
  [ 4.52506632e-01  5.04445970e-01 -1.31325984e+00 ...  3.76044202e+00
   -2.23360211e-02  1.56650240e-05]
  [ 4.65079933e-01  4.76524860e-01 -1.12527049e+00 ...  3.67933726e+00
   -1.00817576e-01  1.78909322e-05]
  ...
  [ 4.85962570e-01  5.14965296e-01 -7.92268634e-01 ...  3.14527488e+00
   -5.84495902e-01  3.82902836e-05]
  [ 4.84566599e-01  5.18746018e-01 -8.30349445e-01 ...  3.14688921e+00
   -5.67315459e-01  3.75931268e-05]
  [ 4.83474344e-01  5.21132410e-01 -8.93516719e-01 ...  3.14198637e+00
   -5.67088902e-01  4.04220882e-05]]

 [[ 5.63206851e-01  5.25112569e-01 -9.14769173e-01 ...  2.98509598e+00
   -4.34871763e-01  7.10284949e-05]
  [ 5.65895200e-01  5.24872720e-01 -9.15838599e-01 ...  2.98157907e+00
   -4.79140371e-01  7.02433899e-05]
  [ 5.69179475e-01  5.16568422e-01 -9.43855107e-01 ...  2.97721338e+00
   -4.92416769e-01  7.15469214e-05]
  ...
  [ 5.61973453e-01  5.2

In [36]:
y = to_categorical(labels).astype(int)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [39]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [40]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,132)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [41]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [42]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 4s 55ms/step - loss: 1.0620 - categorical_accuracy: 0.4588
Epoch 2/2000
3/3 [==============================] - 0s 44ms/step - loss: 1.5052 - categorical_accuracy: 0.3647
Epoch 3/2000
3/3 [==============================] - 0s 43ms/step - loss: 1.0720 - categorical_accuracy: 0.5412
Epoch 4/2000
3/3 [==============================] - 0s 45ms/step - loss: 1.0576 - categorical_accuracy: 0.4706
Epoch 5/2000
3/3 [==============================] - 0s 47ms/step - loss: 1.0178 - categorical_accuracy: 0.3294
Epoch 6/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.9787 - categorical_accuracy: 0.6235
Epoch 7/2000
3/3 [==============================] - 0s 52ms/step - loss: 0.7984 - categorical_accuracy: 0.6000
Epoch 8/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.5585 - categorical_accuracy: 0.8118
Epoch 9/2000
3/3 [==============================] - 0s 43ms/step - loss: 1.6020 - categorical_accuracy: 0.5529
E

In [56]:
res = model.predict(X_test)
actions[np.argmax(res[4])]

1/1 [==============================] - 0s 62ms/step


'iloveyou'

In [44]:
model.save('action.h5')

In [ ]:
del model

In [ ]:
model.load_weights('action.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('pose.tflite', 'wb') as f:
    f.write(tflite_model)

In [45]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [59]:
yhat = model.predict(X_test)
print(X_test.shape)
print(yhat)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

1/1 [==============================] - 0s 405ms/step
(5, 30, 132)
[[1.0000000e+00 2.7102437e-10 8.6181883e-27]
 [1.0241984e-08 1.0000000e+00 4.7788826e-12]
 [0.0000000e+00 3.9095145e-18 1.0000000e+00]
 [8.0108436e-21 2.9745317e-10 1.0000000e+00]
 [1.3820672e-15 6.0445736e-03 9.9395543e-01]]


array([[[4, 0],
        [0, 1]],

       [[3, 0],
        [1, 1]],

       [[2, 1],
        [0, 2]]], dtype=int64)

In [47]:
accuracy_score(ytrue, yhat)

0.8

In [48]:
from scipy import stats

In [49]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [58]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.75

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, pose)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            
            if res[np.argmax(res)] > threshold: 
                
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('action.h5')

# Convert to TensorFlow Lite without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Apply post-conversion quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()